# Рекомендательные системы. Рекомендации через поиск ближайших соседей

In [ ]:
pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 64.7 MB/s eta 0:00:00


## FAISS

> Необходимо сперва установить faiss библиотеку. Инструкцию можно найти [здесь](https://github.com/facebookresearch/faiss/blob/master/INSTALL.md).
```conda install faiss-cpu -c pytorch # CPU version only```

In [ ]:
import numpy as np

In [ ]:
dim = 512  # рассмотрим произвольные векторы размерности 512
nb = 10000  # количество векторов в индексе
nq = 1 # количество векторов в выборке для поиска
np.random.seed(58) # DON't CHANGE THIS
vectors = np.random.random((nb, dim)).astype('float32')
query = np.random.random((nq, dim)).astype('float32')

In [ ]:
vectors.shape

(10000, 512)

In [ ]:
vectors

array([[0.36510557, 0.4512059 , 0.49606034, ..., 0.76589304, 0.00648978,
        0.83173156],
       [0.4488682 , 0.98864484, 0.9459802 , ..., 0.3300889 , 0.31562248,
        0.37883386],
       [0.85288   , 0.15087937, 0.48739725, ..., 0.4501739 , 0.6276719 ,
        0.9150207 ],
       ...,
       [0.5569578 , 0.4969434 , 0.5190067 , ..., 0.04685995, 0.11529469,
        0.6037052 ],
       [0.9910725 , 0.42323965, 0.31499565, ..., 0.8129928 , 0.57980275,
        0.84211975],
       [0.33896688, 0.8154824 , 0.9685506 , ..., 0.32354274, 0.49578118,
        0.55211055]], dtype=float32)

### IndexFlatL2

Создаем Flat индекс и добавляем векторы без обучения

In [ ]:
import faiss


index = faiss.IndexFlatL2(dim)
index.add(vectors)
print(index.ntotal)

10000


In [ ]:
topn = 7

Проведем поиск по нашим векторам из query:

In [ ]:
%%time

D, I = index.search(query, topn)

print(I)
print(D)

[[3214 8794 9507 6591 8728 3959 5485]]
[[70.10199  70.75224  72.443085 72.87785  72.94414  73.420784 73.75675 ]]
CPU times: user 5.76 ms, sys: 0 ns, total: 5.76 ms
Wall time: 4.56 ms


### Inverted File Index

Необходмио создать quantiser(IndexFlatL2), индекс (IndexIVFFlat), обучить индекс и добавить вектора в индекс.

In [ ]:
%%time
k = 10 # количество центроидов

quantiser = faiss.IndexFlatL2(dim)
index = faiss.IndexIVFFlat(quantiser, dim, k)
index.train(vectors)
index.add(vectors)

CPU times: user 234 ms, sys: 28.9 ms, total: 263 ms
Wall time: 353 ms


Необходимо произвести поиск по индексу нашего запроса (query).

In [ ]:
%%time

D, I = index.search(query, topn)
print(I)
print(D)

[[9507 5485 9530 8678 7046 5314 3492]]
[[72.443085 73.75675  74.077194 74.20973  75.07445  75.194534 75.362595]]
CPU times: user 1.74 ms, sys: 38 µs, total: 1.78 ms
Wall time: 1.42 ms


## Применим FAISS для рекомендаций в нашей задаче

Построим простейший рекомендательный сервис.

In [ ]:
import faiss
import numpy as np
import pandas as pd
from scipy.sparse import coo_matrix
from sklearn.decomposition import NMF
from flask import Flask, jsonify, request

# constants
RANDOM_STATE = 57
N_FACTOR = 20 # размерность эмбедингов
N_RESULT = 10 # сколько фильмов рекомендуем

In [ ]:
ratings = pd.read_csv("ratings.csv")
movies = pd.read_csv("movies.csv")

In [ ]:
users = sorted(np.unique(ratings['userId']))
movies = sorted(np.unique(ratings['movieId']))

In [ ]:
# for later use
user_id2i = {id: i for i, id in enumerate(users)}
movie_id2i = {id: i for i, id in enumerate(movies)}
movie_i2id = {i: id for i, id in enumerate(movies)}

In [ ]:
# make sparse matrix
rating_mat = coo_matrix(
    (ratings['rating'], (ratings['userId'].map(user_id2i), ratings['movieId'].map(movie_id2i)))
)

In [ ]:
rating_mat.todense()

matrix([[4. , 0. , 4. , ..., 0. , 0. , 0. ],
        [0. , 0. , 0. , ..., 0. , 0. , 0. ],
        [0. , 0. , 0. , ..., 0. , 0. , 0. ],
        ...,
        [2.5, 2. , 2. , ..., 0. , 0. , 0. ],
        [3. , 0. , 0. , ..., 0. , 0. , 0. ],
        [5. , 0. , 0. , ..., 0. , 0. , 0. ]])

In [ ]:
# decompose
model = NMF(n_components=N_FACTOR, init='random', random_state=RANDOM_STATE)
user_mat = model.fit_transform(rating_mat)
movie_mat = model.components_.T

/usr/local/lib/python3.10/dist-packages/sklearn/decomposition/_nmf.py:1741: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  warnings.warn(


> **NMF** = Non-negative Matrix Factorization. Можно применять метод чередующихся наименьших квадратов (ALS) для неотрицательного матричного разложения. Ключевая идея - искать поочередно то столбцы $p_t$, то столбцы $q_t$ при фиксированных остальных.

In [ ]:
# indexing
# movie_index = faiss.IndexFlatL2(N_FACTOR)


k = 100 # количество центроидов
# необходимо дописать методы
quantiser = faiss.IndexFlatL2(N_FACTOR)
movie_index = faiss.IndexIVFFlat(quantiser, N_FACTOR, k)
movie_index.train(movie_mat.astype('float32'))
movie_index.add(movie_mat.astype('float32'))

In [ ]:
# create app

In [ ]:

def recom_for_user(user_i):
    id = user_id2i[user_i]
    user_vec = user_mat[id]
    scores, indices = movie_index.search(np.array([user_vec]), N_RESULT)


    movie_scores = zip(indices[0], scores[0])
    for index, scr in movie_scores:
      print(int(movie_i2id[index]))


NameError: name 'indices' is not defined

In [ ]:
recom_for_user(128)

59985
81512
3225
32302
1675
4453
5580
55020
3103
7377


> Note: use this link in your browser to acccess your server: http://0.0.0.0:5000/?user_id=128